In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

import remap_values as rv
import good_plots   as gp

import cPickle

In [2]:
train_df = pd.read_csv('inp_data/train.csv')
test_df  = pd.read_csv('inp_data/test.csv')

train_df, test_df = rv.run_tan( train_df, test_df )

remap_values.py:554: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  inds = df[ df['PoolQC'].isnull() ][ df['PoolArea'] > 0 ].index


In [3]:
big_df = train_df.drop( 'SalePrice', axis=1).append( test_df, ignore_index=True)

In [4]:
num_df = train_df.ix[:, train_df.dtypes==float ].copy()
num_df = num_df.join( train_df.ix[:, train_df.dtypes==int ], how='inner'  ).copy()

In [5]:
cat_df = train_df.ix[:, train_df.dtypes=='O'].copy()

In [6]:
qual_cols = ['ExterQual', 'ExterCond', 'GarageQual', 'GarageCond', 'FireplaceQu', 'KitchenQual', 'HeatingQC', 'BsmtQual']
qual_map  = {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}

for col in qual_cols:
    cat_df[col] = cat_df[col].replace( qual_map )

In [7]:
expo_map = {'None': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4}
cat_df['BsmtExposure'] = cat_df['BsmtExposure'].replace( expo_map )

In [8]:
bfin_map = {'None': 1, 'Unf': 1, 'LwQ': 2,'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6}
cat_df.ix[:,['BsmtFinType1','BsmtFinType2']] = cat_df.ix[:,['BsmtFinType1','BsmtFinType2']].replace( bfin_map )

In [9]:
func_map = {'None': 0, 'Sal': 1, 'Sev': 2, 'Maj2': 3, 'Maj1': 4, 'Mod': 5, 'Min2': 6, 'Min1': 7, 'Typ': 8}
cat_df['Functional'] = cat_df['Functional'].replace( func_map )

In [10]:
gfin_map = {'None': 1,'Unf': 1, 'RFn': 1, 'Fin': 2}
cat_df['GarageFinish'] = cat_df['GarageFinish'].replace( gfin_map )

In [11]:
fence_map = {'None': 1, 'MnWw': 1, 'GdWo': 1, 'MnPrv': 2, 'GdPrv': 4}
cat_df['Fence'] = cat_df['Fence'].replace( fence_map )

In [12]:
MSd_map = {'20': 1, '30': 0, '40': 0, '45': 0,'50': 0, '60': 1, '70': 0, '75': 0, '80': 0, 
           '85': 0, '90': 0, '120': 1, '150': 0, '160': 0, '180': 0, '190': 0}

cat_df['NewerDwelling'] = cat_df['MSZoning'].replace( MSd_map )

In [15]:
n = 500
cols = num_df.columns.values
temp_df = num_df.ix[:n,np.insert( cols[:5], 0, 'SalePrice' )].copy()
gp.corr_plot( temp_df )

In [75]:
num_df.corrwith( new_df['SalePrice'] ).sort_values( ascending=False )

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
GarageYrBlt      0.508043
YearRemodAdd     0.507101
MasVnrArea       0.472614
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.334901
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
dtype: float